In [171]:
import pandas as pd
import re
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

## Loading comments

In [172]:
df_bc = pd.read_csv('data/comments/bitcoin.csv')
df_dj = pd.read_csv('data/comments/dj.csv')
df_ts = pd.read_csv('data/comments/tesla.csv')

## Convert date and delete unnecessary tables

In [173]:
from dateutil import parser
from datetime import date
import re

def convert_date(date_string):
    
    if 'ago' in str(date_string):
        today = date.today()
        return today.strftime("%d-%m-%y")
    
    if 'nan' in str(date_string):
        return
    
    dt = parser.parse(date_string)  
    return dt.strftime("%d-%m-%y")

def convert(df):
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df['date'] = df['date'].apply(lambda x: convert_date(x))
    
    return df

In [174]:
df_bc = convert(df_bc)
df_dj = convert(df_dj)
df_ts = convert(df_ts)

C:\Users\cfo19\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1204: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [175]:
del df_ts['+/-']

In [176]:
df_bc = df_bc.dropna(subset=['comment'])
df_dj = df_dj.dropna(subset=['comment'])
df_ts = df_ts.dropna(subset=['comment'])

## Loading price data

In [177]:
df_bcp = pd.read_excel('data/price/BTC_old.xlsx')
df_djp = pd.read_excel('data/price/DJI_old.xlsx')
df_tsp = pd.read_excel('data/price/TSLA_old.xlsx')

 ## Merging price and comment data

In [178]:
df_bc_merged = pd.merge(df_bc,df_bcp, left_on="date", right_on="Date",how="inner")
df_dj_merged = pd.merge(df_dj,df_djp, left_on="date", right_on="Date",how="inner")
df_ts_merged = pd.merge(df_ts,df_tsp, left_on="date", right_on="Date",how="inner")

In [179]:
df_bc_merged = df_bc_merged.dropna(subset=['comment'])
df_dj_merged = df_dj_merged.dropna(subset=['comment'])
df_ts_merged = df_ts_merged.dropna(subset=['comment'])

In [180]:

def add_placeholder(row):
    
    regex1 = r"\b[0-9]{1,}\b"
    regex2 = r"\b[0-9]{1,}\,[0-9]{1,}\b"
    
    #Replace ',' with '.'
    if re.findall(regex2,row['comment']):
        commentwithcomma_list = re.findall(regex2, row['comment'])
        
        for match in commentwithcomma_list:
            match_replaced = match.replace(",",".")
            #print(match_replaced)
            return row['comment'].replace(match,match_replaced)
    #
    if re.findall(regex1,row['comment']):
        predictions_list = re.findall(regex1,row['comment'])
        
        for prediction in predictions_list:    
            predictionVal = float(prediction.replace('$', ''))
    
            if predictionVal < row['Upper Bound'] and predictionVal > row['High']:
                return row['comment'].replace(prediction,' [OPTIMISTICVALUE] ')
            elif predictionVal > row['Lower Bound'] and predictionVal < row['Low']:
                return row['comment'].replace(prediction,' [PESSIMISTICVALUE] ')
            
    return row['comment']

def placeholder(df):
    df.drop(['Date'], axis=1,inplace=True)
    df=df.set_index('date')
    df['updated_comment'] = df.apply(lambda x: add_placeholder(x), axis=1)
    
    return df

In [181]:
df_bc_merged = placeholder(df_bc_merged)
df_dj_merged = placeholder(df_dj_merged)
df_ts_merged = placeholder(df_ts_merged)

In [182]:
df_ts_merged = df_ts_merged.loc[:'02-01-18'] # Alligned data 

## Creating excels for handlabeling

In [183]:
df_bc_merged.to_excel("data/output/bitcoin.xlsx", sheet_name='Sheet_1')
df_dj_merged.to_excel("data/output/dowjones.xlsx", sheet_name='Sheet_1')
df_ts_merged.to_excel("data/output/tesla.xlsx", sheet_name='Sheet_1')

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/al-zf%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20With%20support%20established%20in%20the%205550.%20area,%20longs%20have%20targets%20of%20resistance%20in%20the%205884.%20and%206357.%20areas.%20Stop%20loss%20in%20the%205320.%20area.%20Shorts%20have%20targets%20of%20support%20in%20the%205320.%20and%205194.%20areas.%20Stop%20loss...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/alkkj%20%20%20%20With%20support%20in%20the%205320.%20area%20established%20longs%20have%20targets%20of%20resistance%20in%20the%205550.%20and%206357.%20areas.%20Stop%20loss%20in%20the%205194,%20area.%20Shorts%20have%20targets%20of%20support%20in%20the%205194,%20and%205050.%20ar

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/8uz4s%20top%20USTD%20bottom%20BTC,%20Basically%20if%20atm%20if%20USDT%20stays%20under%20that%20dotted%20line%20and%20BTC%20stays%20above%20it%20has%20the%20flexibility%20to%20go%20up....%20The%20problem%20most%20things%20will%20stay%20in%20range%20cause%20traders%20don't%20generally%20play%20markets%20in%20the%20middle%20of...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://www.tradingview.com/x/I5NC7nPf/%20i%20give%20the%20green%20line%20a%20good%20chance%20to%20survive.%20SL%20moved%20to%206000.%20(faded%20blue%20is%20my%20lambo%20dream%20way%20and%20why%20i%20refused%20to%20sell%20lol)%20adding%20more%20at%20these%20levels
' with link or location/anchor > 255 characters since it exceeds Excel

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7m-cq%20%20%20.%20%20Daily%20BTC%20has%20managed%20to%20close%20within%20the%20%201%20STD%20Dev%20Neutral%20Zone%20for%205%20days%20if%20today%20holds,%20this%20is%20not%20anything%20to%20write%20home%20about%20to%20make%20a%20bullish%20case%20as%20it%20was%20rejected%20@%20the%2020%20SMA,%20and%20worse%20it%20has%20failed%20to%20obtain%20a%20cl...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7m231%20I’m%20still%20very%20short%20term%20bullish.%20%20%207900-8200.%20%20%20%20Here’s%20a%20daily%20ichimoku%20interpretation%20in%20my%20view.%20%20I’ve%20highlighted%20in%20gold%20a%20bearish%20T/K%20cross.%20%20Blue%20crossesdown%20through%20orange.%20And%20because%20this%20cross%20happen

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/6prnu%20Closed%20remaining%20long%20just%20now.%20Having%202%20shorts%20still%20going,%20both%20of%20them%20have%20SL%20in%2011.3k.%20Rising%20wedge%20in%20play%20now,%20lets%20see%20how%20it%20plays%20out.%20If%20the%20wedge%20plays%20fully%20out%20there%20is%20possibility%20to%20have%20big%20H&amp;S;%20pattern.%20That%20would%20be...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'http://blocktivity.info/%20showing%20BTC%20network%20is%20&lt;100%25%20utilization,%20the%20news%20people%20are%20noticing%20this%20too:.%20Bitcoin%20Fees%20Sink%20to%206%20Month%20Low,%20Undermining%20Bitcoin%20Cash.%20Bitcoin%20(BTC)%20transaction%20fees%20have%20dipped%20below%20than%20those%20of%20Bitcoin%20Cash%20(BCH),

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://www.tradingview.com/x/I5NC7nPf/%20i%20give%20the%20green%20line%20a%20good%20chance%20to%20survive.%20SL%20moved%20to%20%20%5bPESSIMISTICVALUE%5d%20.%20(faded%20blue%20is%20my%20lambo%20dream%20way%20and%20why%20i%20refused%20to%20sell%20lol)%20adding%20more%20at%20these%20levels
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://www.zerohedge.com/news/ [PESSIMISTICVALUE] -09-11/he-turned-14502-800-million-now-worlds-biggest-crypto-hedge-fund-hurting?utm_source=feedburner&amp;utm;_medium=feed&amp;utm;_campaign=Feed%3A+zerohedge%2Ffeed+%28zero+hedge+-+on+a+long+enough+timeline%2C+the+survival+rate+fo...Show more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url)

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7od6o%20%20likin'%20these%20short%20time%20frame%20dips%20when%20Gung%20Ho%20%20%5bPESSIMISTICVALUE%5d%20%20hr%204hr%20&amp;%20Daily%20r%20up,%20buy%20n%20sell,%20tight%20stops%20on%20balance%20-%20hate%20being%20long%20when%20I'm%20bearish%20but%20my%20system%20requires%20me%20%20%5bPESSIMISTICVALUE%5d%20%20follow%20the%20tape.
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7na1d%20I’ll%20mention%20this%20%20again%20with%20a%20chart.%20%20At%20%20%5bOPTIMISTICVALUE%5d%20%20area%20it%20became%20apparent%20to%20me%20this%20was%20toppy%20and%20I%20exited%20my%20longs.%20%20%20In%20order%20to%20continue%20any%20bullish%20upwards%20movement%20price%20has%20to%20retrace%20and%20attempt%20a%20higher%20

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://themerkle.com/south-korean-regulators-inch-closer-to-legalizing-bitcoin/?utm_medium=push&amp;utm;_source=onesignal&amp;utm;_campaign=traffic%25%20%5bPESSIMISTICVALUE%5d%20boost&amp;utm;_content=extended%25%20%5bPESSIMISTICVALUE%5d%20%25traffic%25boost
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/6prnu%20Closed%20remaining%20long%20just%20now.%20Having%20%20%5bPESSIMISTICVALUE%5d%20%20shorts%20still%20going,%20both%20of%20them%20have%20SL%20in%2011.3k.%20Rising%20wedge%20in%20play%20now,%20lets%20see%20how%20it%20plays%20out.%20If%20the%20wedge%20plays%20fully%20out%20there%20is%20possibility%20to%20have%20big%20H&amp;S;%20pattern.%20That%20would%20be...Show%20more
' with link or location/anchor > 255 ch

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://www.bloomberg.com/news/articles/2018-05-16/tesla-strikes-lithium-supply-deal-with-australian-mine-developer%20Kidman%20Resources%20Ltd.%20will%20supply%20Tesla%20from%20a%20planned%20refinery%20in%20Western%20Australia%20for%20an%20initial%20term%20of%20three%20years,%20the%20Melbourne-based...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


## Reading data after handlabeling

In [184]:
df_aske=pd.read_excel('data/labeled_data/Aske.xlsx')
df_casper=pd.read_excel('data/labeled_data/Casper.xlsx')
df_morten=pd.read_excel('data/labeled_data/Morten.xlsx')
df_mathias=pd.read_excel('data/labeled_data/Mathias.xlsx')



## Concatinating and cleaning data

In [185]:
df_label = pd.concat([df_aske, df_casper, df_morten, df_mathias], ignore_index=True, sort=True)
del df_label['ID']

In [186]:
def clean_data(df, df_type):
    
    if df_type == "big":
        df=df.drop(['comment', 'High','Low','Upper Bound','Lower Bound'], axis=1, inplace=False)
        df.rename(columns={'updated_comment':'COMMENT\n'}, inplace=True)
        df["LABEL"] = np.nan
    
    if df_type == "big" or "hl":
        
        df['COMMENT\n'] = df['COMMENT\n'].apply(lambda x: re.sub('http://\S+|https://\S+', " ", x))
        df['COMMENT\n'] = df['COMMENT\n'].apply(lambda x: re.sub("\d+", " ", x))
        df['COMMENT\n'] = df['COMMENT\n'].apply(lambda x: re.sub("\d+", " ", x))
        df.rename(columns={'COMMENT\n':'COMMENT'}, inplace=True)
    
    
    return df

OBS - handlabled data has to be passed with hl whereas the merged original comment data has to be passed with big

In [187]:
df_hlabel = clean_data(df_label,"hl")
df_bc = clean_data(df_bc_merged,"big")
df_dj = clean_data(df_dj_merged,"big")
df_ts =clean_data(df_ts_merged,"big")

In [188]:
df_bc.head(10)

,COMMENT,LABEL
date,,
04-05-19,..$ holds and $ [OPTIMISTICVALUE] becomes a ...,NaN
04-05-19,..thumbs down if you nasty bears love getting ...,NaN
04-05-19,Mysterious Crypto Whale Unveils Meteoric Bitco...,NaN
04-05-19,Probably be down [PESSIMISTICVALUE] % by Mond...,NaN
04-05-19,Short BTC for maßsive profits!\r\n,NaN
04-05-19,How many BTC u buy in da dip? A lot\r\n,NaN
04-05-19,Wall Street Analyst Screams ‘Buy Bitcoin’ Befo...,NaN
04-05-19,\r\n,NaN
04-05-19,~ [OPTIMISTICVALUE] should be resistance for ...,NaN



This notebook will now

- Tokenize each sentence
- remove stopwords 
- All characters to lower case
- remove puncation 
- Lemmatization with respect to Part of speech tagging

In [189]:
def preperation(input_str, ls):

    input_str = input_str.lower()
    input_str = input_str.translate(str.maketrans("‘", ",", string.punctuation))
    input_str = input_str.strip()
    
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(input_str)
    result = [i for i in tokens if not i in stop_words]
    

    if ls == 'lemma':
        wnl=WordNetLemmatizer()
        lemma=[]
        for word, tag in pos_tag(result):
            if tag.startswith("NN"):
                lemma.append(wnl.lemmatize(word, pos='n'))
            elif tag.startswith('VB'):
                lemma.append(wnl.lemmatize(word, pos='v'))
            elif tag.startswith('JJ'):
                lemma.append(wnl.lemmatize(word, pos='a'))
            else:
                lemma.append(word)
        return lemma
    
    elif ls == 'stemmer':
        stemmer = SnowballStemmer('english')
        stem = []
        for word in result:
            stem.append(stemmer.stem(word))
        return stem
    #for word in result:
        #print(stemmer.stem(word))
    



In [192]:
def preprocess(df):
    
    
    df['COMMENT']=df['COMMENT'].apply(lambda x: preperation(x,'lemma'))
    df['COMMENT']=df['COMMENT'].apply(lambda x: ", ".join(x))
    df = df[pd.notnull(df['COMMENT'])]
    df = df[df.COMMENT != ""]
    
    return df

In [193]:
df_hlabel = preprocess(df_hlabel)

In [194]:
df_bc = preprocess(df_bc)

In [195]:
df_ts = preprocess(df_ts)

In [196]:
df_dj = preprocess(df_dj)

## Output of labeled data to classifier 

In [197]:
df_label.to_excel("data/output/clean_handlabeled_data.xlsx", sheet_name='Sheet_1') 

In [198]:
df_bc.to_excel("data/output/clean_bit_data.xlsx", sheet_name='Sheet_1') 

In [199]:
df_ts.to_excel("data/output/clean_tesla_data.xlsx", sheet_name='Sheet_1') 

In [200]:
df_dj.to_excel("data/output/clean_dowjones_data.xlsx", sheet_name='Sheet_1') 